In [1]:
#Plotting
import matplotlib.pyplot as plt

# Data analysis
import pandas as pd
import numpy as np
from numpy import array

#Error measurement metrics
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import StandardScaler

## Pytorch libraries to define the LSTM
import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable 

from torch.optim.lr_scheduler import ReduceLROnPlateau

#Import partial autocorrelation plot
from statsmodels.graphics.tsaplots import plot_pacf

#Import UTuning
from UTuning import scorer, plots

# Module to fit gaussian mixtures
from sklearn import mixture

import random

In [2]:
#Set seed numbers for reproducibility
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [3]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def time_array(array):
    """
    Returns a time array in months
    """
    time=np.arange(0,len(array),1)
    return time

def plot_loss(history):
    plt.figure(figsize=(12,6))
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    #plt.plot(history.history['val_loss'], label='val_loss')
    #plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error [Production]')
    plt.legend()
    plt.grid(True) 

In [4]:
def to_tensor(arr):
    tensor_array=Variable(torch.Tensor(arr))
    return tensor_array

def reshape(tensor):
    return torch.reshape(tensor,  (tensor.shape[0], 1, tensor.shape[1])) 

def to_numpy(tensor_array):
    return tensor_array.data.numpy()

In [5]:
def return_ensemble(n_cases,X_test):
    ensemble = np.zeros([cases,X_test.shape[0]])
    for i in range(n_cases):
    
        test_predict = lstm1(X_test)#forward pass
        #data_predict = test_predict.data.numpy() #numpy conversion
        
        ensemble[i,:]= to_numpy(test_predict[:,0])
        
    return ensemble

In [6]:
def return_step(time,difference,window):
    array = np.zeros([1,3])
    array[0,0] = time
    array[0,1] = difference
    array[0,2] = window
    return array

In [7]:
def generate_ensemble(step):
    #X_test_tensors = to_tensor(X_test_step_p50)#Variable(torch.Tensor(X_test))
    X_test_tensor = reshape(to_tensor(step))#torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1])) 
    return return_ensemble(50,X_test_tensor)

In [8]:
def return_percentiles(arr):
    return np.percentile(arr,10),np.percentile(arr,50),np.percentile(arr,90)